### Importing libraries

In [1]:
import pandas as pd

## Reading files

In [2]:
results = pd.read_csv('Data/results.csv')

drivers = pd.read_csv('Data/drivers.csv', encoding='latin-1')
driverStanding = pd.read_csv('Data/driverStandings.csv')

circuits = pd.read_csv('Data/circuits.csv', encoding='latin-1')
races =  pd.read_csv('Data/races.csv')

constructors = pd.read_csv('Data/constructors.csv')
constructorResult = pd.read_csv('Data/constructorResults.csv')
constructorStanding = pd.read_csv('Data/constructorStandings.csv')

pitStops =  pd.read_csv('Data/pitStops.csv')

qualifying = pd.read_csv('Data/qualifying.csv')

weather = pd.read_csv('Data/Weather_SafetyCar.csv')

## Droping and renaming columns 

In [3]:
results.drop(columns=['fastestLapTime', 'fastestLap', 'time', 'positionText', 'position', 'number', 'milliseconds'], inplace=True)

drivers.drop(columns=['number', 'dob', 'nationality', 'url', 'forename', 'surname', 'code'], inplace=True)
driverStanding.drop(columns=['positionText'], inplace=True)

circuits.drop(columns=['alt', 'url', 'location', 'country', 'circuitRef'], inplace=True)
races.drop(columns=['url', 'time'], inplace=True)

constructors.drop(columns=['url', 'Unnamed: 5'], inplace=True)
constructorResult.drop(columns=['status'], inplace=True)
constructorStanding.drop(columns=['positionText', 'Unnamed: 7'], inplace=True)

pitStops.drop(columns=['time'], inplace=True)

qualifying.drop(columns=['q1', 'q2', 'q3'], inplace=True)

weather.drop(columns=['SC Laps'], inplace=True)

In [4]:
results.rename(columns={'points': 'points results'}, inplace=True)

driverStanding.rename(columns={'points': 'points driver standing'}, inplace=True)

circuits.rename(columns={'name': 'circuit name'}, inplace=True)
races.rename(columns={'name': 'race name'}, inplace=True)
weather.rename(columns={'name': 'race name'}, inplace=True)

constructorResult.rename(columns={'points': 'points constructor results'}, inplace=True)
constructorStanding.rename(columns={'points': 'points constructor standing',
                                    'position': 'position constructor standing',
                                    'wins': 'Wins constrcuntor standing'}, inplace=True)

## Merging all the files in one

In [5]:
driver_table = pd.merge(drivers, driverStanding, on='driverId').dropna()
driver_table.drop(columns=['driverRef', 'driverStandingsId'], inplace=True)

race_circuit_table = pd.merge(pd.merge(circuits, races, on='circuitId'), weather, on=['year', 'race name']).dropna()
race_circuit_table.drop(columns=['circuit name', 'round', 'race name'], inplace=True)

constructors_table = pd.merge(pd.merge(constructors, constructorStanding, on='constructorId'), constructorResult, on=['constructorId', 'raceId']).dropna()
constructors_table.drop(columns=['constructorRef', 'name', 'nationality', 'constructorStandingsId', 'constructorResultsId'],  inplace=True)

In [6]:
results_driver_table = pd.merge(driver_table, results, on=['driverId', 'raceId'])

results = pd.merge(results, race_circuit_table, on='raceId')

results = pd.merge(results, constructors_table, on=['constructorId', 'raceId'])

## creating and sorting columns

In [7]:
results['weather_numeric'] = results['weather'].replace({'Dry': 0, 'Wet': 1, 'Varied': 2})
results['race finished'] = results['statusId'].apply(lambda x: 0 if x == 1 else 1)
results = results[['resultId', 'raceId', 'driverId', 'constructorId', 'circuitId', 'statusId', 'grid', 'positionOrder',
                   'points results', 'laps', 'rank', 'fastestLapSpeed', 'lat', 'lng', 'date', 'year', 'weather',
                   'weather_numeric', 'points constructor standing', 'position constructor standing',
                   'Wins constrcuntor standing', 'points constructor results', 'race finished']]

## Exporting data to csv

In [8]:
results.to_csv('total_data/results.csv', index=False)